In [1]:
import os

In [2]:
%pwd

'c:\\Users\\nikhi\\End-To-End-ML-Project\\research'

In [3]:
os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import List, Optional

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str

    n_estimators: List[int]
    max_depth: List[Optional[int]]
    min_samples_split: List[int]
    min_samples_leaf: List[int]
    max_features: List[str]

    target_column: str

In [10]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,

            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            min_samples_split=params.min_samples_split,
            min_samples_leaf=params.min_samples_leaf,
            max_features=params.max_features,

            target_column=schema.name
        )

        return model_trainer_config

In [ ]:
import pandas as pd
import os
import joblib

from mlProject import logger
from mlProject.entity.config_entity import ModelTrainerConfig

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)

        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        logger.info("Data loaded and split into train/test sets")

        rf = RandomForestRegressor(
            random_state=42,
            n_jobs=-1
        )

        param_dist = {
            "n_estimators": self.config.n_estimators,
            "max_depth": self.config.max_depth,
            "min_samples_split": self.config.min_samples_split,
            "min_samples_leaf": self.config.min_samples_leaf,
            "max_features": self.config.max_features
        }

        rf_search = RandomizedSearchCV(
            estimator=rf,
            param_distributions=param_dist,
            n_iter=30,
            cv=5,
            scoring="neg_root_mean_squared_error",
            verbose=2,
            random_state=42,
            n_jobs=-1
        )

        logger.info("Starting RandomForest hyperparameter tuning")
        rf_search.fit(train_x, train_y)

        best_model = rf_search.best_estimator_

        logger.info(f"Best RandomForest params: {rf_search.best_params_}")
        logger.info(f"Best CV RMSE: {-rf_search.best_score_}")

        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(best_model, model_path)

        logger.info(f"Best model saved at: {model_path}")

In [18]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2026-01-23 11:40:01,572: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-23 11:40:01,599: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-23 11:40:01,605: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-01-23 11:40:01,609: INFO: common: created directory at: artifacts]
[2026-01-23 11:40:01,611: INFO: common: created directory at: artifacts/model_trainer]
